# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [31]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.evaluation import ClusteringEvaluator

In [32]:
spark = SparkSession.builder.appName('cluster_consult').getOrCreate()
spark

In [33]:
df = spark.read.csv('hack_data.csv',inferSchema=True,header=True)
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [34]:
df.count()

334

In [35]:
df.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [36]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [37]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [38]:
# indexer = StringIndexer(inputCol='Location',outputCol='LocationIndex')
# indexer_model = indexer.fit(df)
# indexer_model

In [39]:
# output = indexer_model.transform(df)
# output.printSchema()

In [40]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'],outputCol='features')
assembler

VectorAssembler_44374b740baa

In [41]:
final_data = assembler.transform(df)
final_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|
|                    2.0|           228.08|              1|             2.48|     

In [42]:
scaler = StandardScaler(inputCol='features',outputCol='scaled_features')
scaler_model = scaler.fit(final_data)
final_data = scaler_model.transform(final_data)
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)



Let's first check for a KMeans model with 3 clusters to see if there were 3 hackers involved in the attack.

In [43]:
kmeans = KMeans().setFeaturesCol('scaled_features').setK(3).setSeed(48)
kmeans_model = kmeans.fit(final_data)
kmeans_model

KMeansModel: uid=KMeans_5cc963663eda, k=3, distanceMeasure=euclidean, numFeatures=6

In [44]:
cluster_centers = kmeans_model.clusterCenters()
cluster_centers

[array([1.21780112, 1.37901802, 1.99757683, 1.37198977, 2.55237797,
        5.29152222]),
 array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.30217042, 1.25830099, 0.        , 1.35793211, 2.57251009,
        5.24230473])]

In [45]:
sse = kmeans_model.summary.trainingCost
sse

434.1492898715821

In [46]:
kmeans_model.summary.predictions.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|     scaled_features|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|         0|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|         2|
|                   31.0|           356.32|              1|             3.7

In [47]:
results = kmeans_model.transform(final_data)
results.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- prediction: integer (nullable = false)



In [48]:
results.select('prediction').show(5)

+----------+
|prediction|
+----------+
|         0|
|         2|
|         0|
|         0|
|         2|
+----------+
only showing top 5 rows



In [49]:
results.select('prediction').distinct().collect()

[Row(prediction=1), Row(prediction=2), Row(prediction=0)]

In [50]:
results.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   84|
|         0|   83|
+----------+-----+



In [51]:
clusteval = ClusteringEvaluator(featuresCol='scaled_features',predictionCol='prediction')
silhouette_score = clusteval.evaluate(results)
print("Silhouette Score of KMeans:",silhouette_score)

Silhouette Score of KMeans: 0.755239186655932


In [52]:
final_data = final_data.drop('prediction')
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)



Let's evaluate a K Means model with 2 clusters to determine if there were only 2 hackers involved in the attack.

In [53]:
kmeans = KMeans(k=2,seed=48,featuresCol='scaled_features')
kmeans_model = kmeans.fit(final_data)
kmeans_model

KMeansModel: uid=KMeans_076f16744221, k=2, distanceMeasure=euclidean, numFeatures=6

In [54]:
cluster_centers = kmeans_model.clusterCenters()
cluster_centers

[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
        5.26676612])]

In [55]:
sse = kmeans_model.summary.trainingCost
sse

601.7707512676691

In [56]:
kmeans_model.transform(final_data).select('scaled_features','prediction').show()

+--------------------+----------+
|     scaled_features|prediction|
+--------------------+----------+
|[0.56785108466505...|         1|
|[1.41962771166263...|         1|
|[2.20042295307707...|         1|
|[0.14196277116626...|         1|
|[1.41962771166263...|         1|
|[0.07098138558313...|         1|
|[1.27766494049636...|         1|
|[1.56159048282889...|         1|
|[1.06472078374697...|         1|
|[0.85177662699757...|         1|
|[1.06472078374697...|         1|
|[2.27140433866020...|         1|
|[1.63257186841202...|         1|
|[0.63883247024818...|         1|
|[1.91649741074455...|         1|
|[0.85177662699757...|         1|
|[1.49060909724576...|         1|
|[0.70981385583131...|         1|
|[1.41962771166263...|         1|
|[1.56159048282889...|         1|
+--------------------+----------+
only showing top 20 rows



In [57]:
results = kmeans_model.transform(final_data)
results.sample(withReplacement=False,fraction=0.2,seed=48).select('prediction').show(15)

+----------+
|prediction|
+----------+
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 15 rows



In [58]:
results.select('prediction').distinct().collect()

[Row(prediction=1), Row(prediction=0)]

In [59]:
results.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [60]:
print("Silhouette Score for a K Means model with 2 clusters:", clusteval.evaluate(kmeans_model.transform(final_data)))

Silhouette Score for a K Means model with 2 clusters: 0.817646009401248


Based on the results, we can conclude that there were only 2 hackers involved in the attack. There were 334 attacks in total out of which each of those hackers traded 167 attacks. This is proved by the higher value of Silhouette coefficient in the case of the K Means model with 2 clusters as compared to the one with 3 clusters.